In [ ]:
import speech_recognition as sr
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import asyncio

r = sr.Recognizer()
app = FastAPI()

origins = [
    "http://localhost:3000",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,  # Cho phép các nguồn gốc được xác định
    allow_credentials=True,
    allow_methods=["*"],  # Cho phép tất cả các phương thức (GET, POST, PUT, DELETE)
    allow_headers=["*"],  # Cho phép tất cả các headers
)

@app.get("/")
async def read_root():
    return {"Hello": "World"}

@app.post("/convertAudio")
async def convert_audio_to_text(file: UploadFile = File(...)):
    with open("audio.wav", "wb") as audio:
        audio.write(file.file.read())
    with sr.AudioFile("audio.wav") as source:
        audio = r.record(source)
    try:
        text = r.recognize_google(audio)
    except sr.UnknownValueError:
        text = "Could not understand the audio"
    except sr.RequestError as e:
        text = "Could not request results; {0}".format(e)
    return {"text": text}

@app.post("/getImg")
async def get_img(file: UploadFile = File(...)):
    with open("img.jpg", "wb") as img:
        img.write(file.file.read())
    return True
        
if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()